# Project Overview

**Project Introduction**

Lightweight fine-tuning is one of the most important techniques for adapting foundation models, because it allows you to modify foundation models for your needs without needing substantial computational resources.

In this project, you will apply parameter-efficient fine-tuning using the Hugging Face `peft` library.

**Project Summary**

In this project, you will bring together all of the essential components of a PyTorch + Hugging Face training and inference process. Specifically, you will:

1. Load a pre-trained model and evaluate its performance
2. Perform parameter-efficient fine tuning using the pre-trained model
3. Perform inference using the fine-tuned model and compare its performance to the original model

## HuggingFace PEFT Library

**Key Concepts**

Hugging Face PEFT allows you to fine-tune a model without having to fine-tune all of its parameters.

Training a model using Hugging Face PEFT requires two additional steps beyond traditional fine-tuning:

1. Creating a **PEFT config**
2. **Converting the model into a PEFT model** using the PEFT config
Inference using a PEFT model is almost identical to inference using a non-PEFT model. The only difference is that it must be loaded as a PEFT model.

## Training with PEFT

Creating a PEFT Config
The PEFT config specifies the adapter configuration for your parameter-efficient fine-tuning process. The base class for this is a `PeftConfig`, but this example will use a `LoraConfig`, the subclass used for low rank adaptation (LoRA).

A LoRA config can be instantiated like this:

```python
from peft import LoraConfig
config = LoraConfig()
```

Look at the LoRA adapter documentation for additional hyperparameters that can be specified by passing arguments to `LoraConfig()`. [Hugging Face LoRA conceptual guide](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora) also contains additional explanations.

## Converting a Transformers Model into a PEFT Model

Once you have a PEFT config object, you can load a Hugging Face `transformers` model as a PEFT model by first loading the pre-trained model as usual (here we load GPT-2):

```python
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")
```

Then using `get_peft_model()` to get a trainable PEFT model (using the LoRA config instantiated previously):

```python
from peft import get_peft_model
lora_model = get_peft_model(model, config)
```

## Training with a PEFT Model

After calling `get_peft_model()`, you can then use the resulting `lora_model` in a training process of your choice (PyTorch training loop or Hugging Face `Trainer`).

## Checking Trainable Parameters of a PEFT Model

A helpful way to check the number of trainable parameters with the current config is the `print_trainable_parameters()` method:

```python
lora_model.print_trainable_parameters()
```

Which prints an output like this:

`trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364`

## Saving a Trained PEFT Model

Once a PEFT model has been trained, the standard Hugging Face `save_pretrained()` method can be used to save the weights locally. For example:

```python
lora_model.save_pretrained("gpt-lora")
```

Note that this **only saves the adapter weights** and not the weights of the original Transformers model. Thus the size of the files created will be much smaller than you might expect.

# Inference with PEFT

## Loading a Saved PEFT Model

Because you have only saved the adapter weights and not the full model weights, you can't use `from_pretrained()` with the regular Transformers class (e.g., `AutoModelForCausalLM`). Instead, you need to use the PEFT version (e.g., `AutoPeftModelForCausalLM`). For example:

```python
from peft import AutoPeftModelForCausalLM
lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora")
```

After completing this step, you can proceed to use the model for inference.

# Generating Text from a PEFT Model

You may see examples from regular Transformer models where the input IDs are passed in as a positional argument (e.g., `model.generate(input_ids)`). For a PEFT model, they must be passed in as a keyword argument (e.g., `model.generate(input_ids=input_ids)`). For example:

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer.batch_decode(outputs))
```

**Documentation Links**

* [Hugging Face PEFT configuration](https://huggingface.co/docs/peft/package_reference/config)
* [Hugging Face LoRA adapter](https://huggingface.co/docs/peft/package_reference/lora)
* [Hugging Face Models save_pretrained](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.PreTrainedModel.save_pretrained)
* [Hugging Face Text Generation](https://huggingface.co/docs/transformers/main_classes/text_generation)

# Project Instructions

To pass this project, your code must:

1. Load a pre-trained model and evaluate its performance
2. Perform parameter-efficient fine-tuning using the pre-trained model
3. Perform inference using the fine-tuned model and compare its performance to the original model

## Getting Started

This project is failry open-ended. As long as you follow the prescribed steps **you may choose any appropriate PEFT technique, model, evaluation approach, and fine-tuning dataset.**

* **PEFT Technique**:

> * The PEFT technique covered in this course was LoRA, but new techniques are continuously being developed. See the [PEFT README](https://github.com/huggingface/peft) for links to the papers behind each of the supported techniques.
> * If you are unsure, we recommend using **LoRA** as your PEFT technique. LoRA is the only PEFT technique that is compatible with all models at this time.

* **Model**

> * Your choice of model will depend on your choice of PEFT technique.
> * Unless you plan to use your own hardware/GPU rather than the Udacity Workspace, it's best to choose a smaller model.
> * The model must be compatible with a sequence classification task.
> * If you are unsure, we recommend using GPT-2 as your model. This is a relatively small model that is compatible with sequence classification and LoRA.

For specific model names in the Hugging Face registry, you can use the widget at the bottom of the [PEFT documentation homepage](https://huggingface.co/docs/peft/index) (select "sequence classification" from the drop-down).

* **Evaluation Approach**

> * The evaluation approach covered in this course was the `evaluate` method with a Hugging Face `Trainer`. You may use the same approach, or any other reasonable evaluation approach for a sequence classification task
> * The key requirement for the evaluation is that you must be able to compare the original foundation model's performance and the fine-tuned model's performance.

* **Dataset**

> * Your PEFT process must use a dataset from Hugging Face's `datasets` library. As with the selection of model, you will need to ensure that the dataset is small enough that it is usable for your workspace
> * The key requirement for the dataset is that it matches the task. Follow this link to [view Hugging Face datasets filtered by the text classification task](https://huggingface.co/datasets?task_categories=task_categories:text-classification)

## Loading and Evaluating a Foundation Model

### Loading the model

Once you have selected a model, load it in your notebook.

### Evaluating the model

Perform an initial evaluation of the model on your chosen sequence classification task. This step will require that you also load an appropriate tokenizer and dataset.

## Performing Parameter-Efficient Fine-Tuning

### Creating a PEFT config

Create a PEFT config with appropriate hyperparameters for your chosen model.

### Creating a PEFT model

Using the PEFT config and foundation model, create a PEFT model.

### Training the model

Using the PEFT model and dataset, run a training loop with at least one epoch.

### Saving the trained model

Depending on your training loop configuration, your PEFT model may have already been saved. If not, use `save_pretrained` to save your progress

## Performing Inference with a PEFT Model

## Loading the model

Using the appropriate PEFT model class, load your trained model.

## Evaluating the model

Repeat the previous evaluation process, this time using the PEFT model. Compare the results to the results from the original foundation model.